In [6]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [7]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

In [8]:
#count number of related items
c = 0
for key, value in nary.items():
    if value.get('children') != None:
        for item in value['children']:
            if item['relation'] == 'is_a':
                c+=1
print(c)

86683


In [9]:
for key, value in nary.items():
    if value['name'] == 'biological_process':
        print(value)

{'id': 'GO:0008150', 'isObsolete': False, 'name': 'biological_process', 'definition': {'text': 'A biological process represents a specific objective that the organism is genetically programmed to achieve. Biological processes are often described by their outcome or ending state, e.g., the biological process of cell division results in the creation of two daughter cells (a divided cell) from a single parent cell. A biological process is accomplished by a particular set of molecular functions carried out by specific gene products (or macromolecular complexes), often in a highly regulated manner and in a particular temporal sequence.'}, 'comment': 'Note that, in addition to forming the root of the biological process ontology, this term is recommended for use for the annotation of gene products whose biological process is unknown. When this term is used for annotation, it indicates that no information was available about the biological process of the gene product annotated as of the date t

In [10]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

In [11]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process(Thing):
        namespace = onto
    class molecular_function(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [12]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [13]:
#remove anything that's not a biological process or molecular function from kilnary

c = 0
keys = []
for key, value in kilnary.items():
    if value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
print(c)
for i in keys:
    del kilnary[i]

1522


In [14]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'negatively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
    if value['aspect'] == 'molecular_function':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'negatively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)


In [15]:
#make bp and mf classes

for key, value in nary.items():

    if value.get('aspect') != None and value['aspect'] == 'biological_process':
        with onto:
            classname = types.new_class(key, (Thing,))
    if value.get('aspect') != None and value['aspect'] == 'molecular_function':
        with onto:
            classname = types.new_class(key, (Thing,))

In [16]:
#make classes 1 level inside bp and mf classes



In [17]:
#assign appropriate labels to classes

x = list(onto.classes())
for y in x:
    term = 'GO:' + str(y)[8:]
    if str(y)[7] == ':':
        y.label = nary[str(y)[5:]]['name']
    if str(y)[7] == 'A':
        y.label = 'anti_' + nary[term]['name'] + '_agent'
    
    if str(y)[7] == 'P':
        y.label = 'pro_' + nary[term]['name'] + '_agent'

In [18]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = cl[5:7] +':' + cl[8:]
    ids.append(clt)

In [19]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto:
            classname = types.new_class((pname), (ObjectProperty,), )
    return 0

In [20]:
#parsing biological process and molecular function classes

reldict = {}

for key, value in kilnary.items():            
    for item in value['children']:
        if reldict.get(item['relation']) == None:
            reldict[item['relation']] = [[],[]]
            propertymaker(item['relation'], 'void', 'void')
    for item in value['children']:
        if item['id'] in ids:
            reldict[item['relation']][1].append(ids[ids.index(value['id'])])
            reldict[item['relation']][0].append(ids[ids.index(item['id'])])

In [21]:
#converting string to class object in reldict lists

for key, value in reldict.items():
    for item in value[0]:
        value[0][value[0].index(item)] = getattr(onto, item)
    for item in value[1]:
        value[1][value[1].index(item)] = getattr(onto, item)


In [22]:
ded = {}

for key, value in reldict.items():
    for item in value[0]:
        ded[value[0][value[0].index(item)]] = {}
        ded[value[0][value[0].index(item)]][key] = []

In [23]:
for key, value in reldict.items():
    for item in value[0]:
        if ded[value[0][value[0].index(item)]].get(key) != None:
            ded[value[0][value[0].index(item)]][key].append(value[1][value[0].index(item)])

In [24]:
#listing positively_regulates and negatively_regulates as subproperties

with onto:
    class positively_regulates(onto.regulates):
        pass
    class negatively_regulates(onto.regulates):
        pass

In [25]:
#setting relations

for key, value in ded.items():
    for key2, value2 in value.items():
        
        setattr(key, key2, value2)

In [26]:
onto.save(file = "testowl4", format = "rdfxml")